In [1]:
from qiskit import BasicAer
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit_optimization.algorithms import (
    MinimumEigenOptimizer,
    RecursiveMinimumEigenOptimizer,
    SolutionSample,
    OptimizationResultStatus,
)
from qiskit_optimization import QuadraticProgram
from qiskit.visualization import plot_histogram
from typing import List, Tuple
import numpy as np
import pickle

In [2]:
# create a QUBO
qubo = QuadraticProgram()
for i in range(9):
    st = 'x{}'.format(i+1)
    qubo.binary_var(st)
    st = 'y{}'.format(i+1)
    qubo.binary_var(st)
    st = 'z{}'.format(i+1)
    qubo.binary_var(st)

In [3]:
with open('Graph_data.pkl', 'rb') as f:
    gdata = pickle.load(f)
edge_lists = gdata['edge_list']

In [4]:
import networkx as nx
import scipy.sparse

G = nx.DiGraph()
G.add_nodes_from(range(9))
G.add_weighted_edges_from(edge_lists)

nodelist = list(G)
A = nx.to_scipy_sparse_matrix(G, nodelist=nodelist, weight="weight", format="csr")
n, m = A.shape
diags = A.sum(axis=0)  # 1 = outdegree, 0 = indegree
D = scipy.sparse.spdiags(diags.flatten(), [0], m, n, format="csr")

L = (A - D).todense()
# print((A - D).todense())
# [[-2  1  1]

coeff = {}
for i in range(9):
    for j in range(9):
        coeff['x{}x{}'.format(i+1,j+1)] = L[i,j] + 1
        coeff['y{}y{}'.format(i+1,j+1)] = L[i,j] + 1
        coeff['z{}z{}'.format(i+1,j+1)] = L[i,j] + 1
        if i == j:
            coeff['x{}x{}'.format(i+1,j+1)] += 1
            coeff['y{}y{}'.format(i+1,j+1)] += 1
            coeff['z{}z{}'.format(i+1,j+1)] += 1
            
        coeff['x{}y{}'.format(i+1,i+1)] = 2
        coeff['x{}z{}'.format(i+1,i+1)] = 2
        coeff['y{}z{}'.format(i+1,i+1)] = 2

In [5]:
quad = {}
for k in list(coeff.keys()):
    xvar1 = k[:2]
    xvar2 = k[2:]
    quad[(xvar1, xvar2)] = coeff[k]

In [6]:
# quadz

In [7]:
qubo.minimize(linear=[-8 for i in range(9*3)], quadratic=quad)
print(qubo.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: - 8 x1 - 8 y1 - 8 z1 - 8 x2 - 8 y2 - 8 z2 - 8 x3 - 8 y3 - 8 z3 - 8 x4
      - 8 y4 - 8 z4 - 8 x5 - 8 y5 - 8 z5 - 8 x6 - 8 y6 - 8 z6 - 8 x7 - 8 y7
      - 8 z7 - 8 x8 - 8 y8 - 8 z8 - 8 x9 - 8 y9 - 8 z9 + [ 4 x1^2 + 4 x1*y1
      + 4 x1*z1 + 4.379531204700 x1*x2 + 4.475295513868 x1*x3
      + 4.104782104492 x1*x4 + 4 x1*x5 + 4 x1*x6 + 4 x1*x7 + 4 x1*x8 + 4 x1*x9
      + 4 y1^2 + 4 y1*z1 + 4.379531204700 y1*y2 + 4.475295513868 y1*y3
      + 4.104782104492 y1*y4 + 4 y1*y5 + 4 y1*y6 + 4 y1*y7 + 4 y1*y8 + 4 y1*y9
      + 4 z1^2 + 4.379531204700 z1*z2 + 4.475295513868 z1*z3
      + 4.104782104492 z1*z4 + 4 z1*z5 + 4 z1*z6 + 4 z1*z7 + 4 z1*z8 + 4 z1*z9
      + 3.620468795300 x2^2 + 4 x2*y2 + 4 x2*z2 + 4 x2*x3 + 4 x2*x4
      + 3.831650286913 x2*x5 + 4.041176848114 x2*x6 + 4.120993800461 x2*x7
      + 4.045388072729 x2*x8 + 4.007385268342 x2*x9 + 3.620468795300 y2^2
      + 4 y2*z2 + 4 y2*y3 + 4

In [8]:
op, offset = qubo.to_ising()
print("offset: {}".format(offset))
print("operator:")
print(op)

offset: -13.60551369987661
operator:
-2.0237932188319974 * ZIIIIIIIIIIIIIIIIIIIIIIIIII
- 2.0237932188319974 * IZIIIIIIIIIIIIIIIIIIIIIIIII
+ 0.5 * ZZIIIIIIIIIIIIIIIIIIIIIIIII
- 2.0237932188319974 * IIZIIIIIIIIIIIIIIIIIIIIIIII
+ 0.5 * ZIZIIIIIIIIIIIIIIIIIIIIIIII
+ 0.5 * IZZIIIIIIIIIIIIIIIIIIIIIIII
- 2.0304418168962 * IIIZIIIIIIIIIIIIIIIIIIIIIII
+ 0.5 * ZIIZIIIIIIIIIIIIIIIIIIIIIII
- 2.0304418168962 * IIIIZIIIIIIIIIIIIIIIIIIIIII
+ 0.5 * IZIIZIIIIIIIIIIIIIIIIIIIIII
+ 0.5 * IIIZZIIIIIIIIIIIIIIIIIIIIII
- 2.0304418168962 * IIIIIZIIIIIIIIIIIIIIIIIIIII
+ 0.5 * IIZIIZIIIIIIIIIIIIIIIIIIIII
+ 0.5 * IIIZIZIIIIIIIIIIIIIIIIIIIII
+ 0.5 * IIIIZZIIIIIIIIIIIIIIIIIIIII
- 2.022300618700683 * IIIIIIZIIIIIIIIIIIIIIIIIIII
+ 0.5 * ZIIIIIZIIIIIIIIIIIIIIIIIIII
+ 0.5 * IIIZIIZIIIIIIIIIIIIIIIIIIII
- 2.022300618700683 * IIIIIIIZIIIIIIIIIIIIIIIIIII
+ 0.5 * IZIIIIIZIIIIIIIIIIIIIIIIIII
+ 0.5 * IIIIZIIZIIIIIIIIIIIIIIIIIII
+ 0.5 * IIIIIIZZIIIIIIIIIIIIIIIIIII
- 2.022300618700683 * IIIIIIIIZIIIIIIIIIIIIIIIIII
+ 0.5 * IIZII

In [9]:
qp = QuadraticProgram()
qp.from_ising(op, offset, linear=True)
print(qp.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: - 6 x0 - 6 x1 - 6 x2 - 6.189765602350 x3 - 6.189765602350 x4
      - 6.189765602350 x5 - 6.237647756934 x6 - 6.237647756934 x7
      - 6.237647756934 x8 - 6.052391052246 x9 - 6.052391052246 x10
      - 6.052391052246 x11 - 5.896891564131 x12 - 5.896891564131 x13
      - 5.896891564131 x14 - 6.070131575223 x15 - 6.070131575223 x16
      - 6.070131575223 x17 - 5.910797525197 x18 - 5.910797525197 x19
      - 5.910797525197 x20 - 5.878232732415 x21 - 5.878232732415 x22
      - 5.878232732415 x23 - 5.904827124672 x24 - 5.904827124672 x25
      - 5.904827124672 x26 + [ 4 x0*x1 + 4 x0*x2 + 4.379531204700 x0*x3
      + 4.475295513868 x0*x6 + 4.104782104492 x0*x9 + 4 x0*x12 + 4 x0*x15
      + 4 x0*x18 + 4 x0*x21 + 4 x0*x24 + 4 x1*x2 + 4.379531204700 x1*x4
      + 4.475295513868 x1*x7 + 4.104782104492 x1*x10 + 4 x1*x13 + 4 x1*x16
      + 4 x1*x19 + 4 x1*x22 + 4 x1*x25 + 4.379531204700 x2*x5
     

In [13]:
from qiskit import IBMQ
from qiskit.providers.ibmq import least_busy
ibm_token = '49670f985b02b984d52967234a8d86aa4f49519731f6a3b1b9b5c2afffde9d8f34d46e6df77cf8ec96e1adf36aadeaf8a6c4a73665cfb2945627ed954451f473' # https://quantum-computing.ibm.com/account
ibmq_account = IBMQ.enable_account(ibm_token)
ibmq_provider = IBMQ.get_provider(hub='ibm-q-skku', group='snu', project='snu-graduate')
ibmq_backend = ibmq_provider.get_backend('ibmq_qasm_simulator')

In [14]:
algorithm_globals.random_seed = 10598
algorithm_globals.massive=True
quantum_instance = QuantumInstance(
    ibmq_backend,
    seed_simulator=algorithm_globals.random_seed,
    seed_transpiler=algorithm_globals.random_seed,
)
qaoa_mes = QAOA(quantum_instance=quantum_instance, initial_point=[0.0, 0.0])
exact_mes = NumPyMinimumEigensolver()

In [15]:
qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA
exact = MinimumEigenOptimizer(exact_mes)  # using the exact classical numpy minimum eigen solver

In [16]:
qaoa_result = qaoa.solve(qubo)
print(qaoa_result)

optimal function value: -34.40323443408124
optimal value: [0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0.
 0. 1. 0.]
status: SUCCESS


In [33]:
c1 = np.array([qaoa_result.x[3*i] for i in range(9)], dtype=int)
c1

array([0, 0, 1, 0, 1, 0, 0, 1, 0])

In [34]:
c2 = np.array([qaoa_result.x[3*i+1] for i in range(9)], dtype=int)
c2

array([1, 0, 0, 1, 0, 0, 1, 0, 1])

In [35]:
c3 = np.array([qaoa_result.x[3*i+2] for i in range(9)], dtype=int)
c3

array([0, 1, 0, 0, 0, 1, 0, 0, 0])

In [ ]:
exact_result = exact.solve(qubo)
print(exact_result)

In [ ]:
print("variable order:", [var.name for var in qaoa_result.variables])
for s in qaoa_result.samples:
    print(s)

In [ ]:
def get_filtered_samples(
    samples: List[SolutionSample],
    threshold: float = 0,
    allowed_status: Tuple[OptimizationResultStatus] = (OptimizationResultStatus.SUCCESS,),
):
    res = []
    for s in samples:
        if s.status in allowed_status and s.probability > threshold:
            res.append(s)

    return res

In [ ]:
filtered_samples = get_filtered_samples(
    qaoa_result.samples, threshold=0.005, allowed_status=(OptimizationResultStatus.SUCCESS,)
)
for s in filtered_samples:
    print(s)

In [ ]:
fvals = [s.fval for s in qaoa_result.samples]
probabilities = [s.probability for s in qaoa_result.samples]

In [ ]:
np.mean(fvals)

In [ ]:
np.std(fvals)

In [ ]:
samples_for_plot = {
    " ".join(f"{qaoa_result.variables[i].name}={int(v)}" for i, v in enumerate(s.x)): s.probability
    for s in filtered_samples
}
samples_for_plot

In [ ]:
plot_histogram(samples_for_plot)

In [ ]:
rqaoa = RecursiveMinimumEigenOptimizer(qaoa, min_num_vars=1, min_num_vars_optimizer=exact)

In [ ]:
rqaoa_result = rqaoa.solve(qubo)
print(rqaoa_result)

In [ ]:
filtered_samples = get_filtered_samples(
    rqaoa_result.samples, threshold=0.005, allowed_status=(OptimizationResultStatus.SUCCESS,)
)

In [ ]:
samples_for_plot = {
    " ".join(f"{rqaoa_result.variables[i].name}={int(v)}" for i, v in enumerate(s.x)): s.probability
    for s in filtered_samples
}
samples_for_plot

In [ ]:
plot_histogram(samples_for_plot)